In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


%matplotlib inline

### Reading in, filtering, and examining the heart attack cost disparities data

In [2]:
# your path to the data file may vary!

ha_costs_df = pd.read_csv('../data/mmd_heart_attack_data.csv') 
tn_ha_costs = ha_costs_df.loc[ha_costs_df.state == 'TENNESSEE']
print(tn_ha_costs.shape)
print(tn_ha_costs.head(2))

(91, 17)
      year geography             measure         adjustment      analysis  \
2030  2017    County  Average total cost  Unsmoothed actual  Base measure   
2031  2017    County  Average total cost  Unsmoothed actual  Base measure   

                          domain                    condition primary_sex  \
2030  Primary chronic conditions  Acute myocardial infarction         All   
2031  Primary chronic conditions  Acute myocardial infarction         All   

     primary_age     primary_dual   fips           county      state  urban  \
2030         All  Dual & non-dual  47001  Anderson County  TENNESSEE  Urban   
2031         All  Dual & non-dual  47003   Bedford County  TENNESSEE  Rural   

     primary_race primary_denominator  analysis_value  
2030          All           undefined           42749  
2031          All           undefined           43661  


### Now getting the cancer data

In [3]:
cancer_costs_df = pd.read_csv('../data/mmd_cancer_data.csv')
tn_cancer_costs = cancer_costs_df.loc[cancer_costs_df.state == 'TENNESSEE']
print(tn_cancer_costs.shape)
print(tn_cancer_costs.head(2))

(95, 17)
      year geography             measure         adjustment      analysis  \
2396  2017    County  Average total cost  Unsmoothed actual  Base measure   
2397  2017    County  Average total cost  Unsmoothed actual  Base measure   

                          domain                                   condition  \
2396  Primary chronic conditions  Cancer, Colorectal, Breast, Prostate, Lung   
2397  Primary chronic conditions  Cancer, Colorectal, Breast, Prostate, Lung   

     primary_sex primary_age     primary_dual   fips           county  \
2396         All         All  Dual & non-dual  47001  Anderson County   
2397         All         All  Dual & non-dual  47003   Bedford County   

          state  urban primary_race primary_denominator  analysis_value  
2396  TENNESSEE  Urban          All           undefined           15454  
2397  TENNESSEE  Rural          All           undefined           19219  


### Getting the income data and cleaning it a bit

In [4]:
income_df = pd.read_csv('../data/irs_county_2016.csv')
tn_income = income_df.loc[income_df.STATE == 'TN']
tn_income.head(2)

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
19752,47,TN,0,Tennessee,1,39580,22400,14440,980,25390,...,6760,17669,200,287,0,0,3970,7167,16170,59567
19753,47,TN,0,Tennessee,2,439770,345410,39920,49300,188490,...,109250,62045,0,0,0,0,37380,18477,366150,366510


In [5]:
tn_income = tn_income[['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']]
tn_income.columns = ['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']


### Week two coding tasks
#### Replacing coded values in the `income_bucket` column with descriptive text
- create a dictionary mapping codes to descriptions
- use `replace()` to update the df with text

In [6]:
income_dict = {0:'Total', 1: 'Under $1', 2: 'Between 1 and $10,000', 3: 'Between 10,000 and $25,000',
              4: 'Between 25,000 and $50,000', 5: 'Between 50,000 and $75,000', 
               6: 'Between 75,000 and $100,000', 7: 'Between 100,000 and $200,000', 
               8:'$200,000 or more'}

In [7]:
tn_income.income_bucket = tn_income.income_bucket.replace(income_dict)
tn_income.head(2)

,state,county,income_bucket,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp
19752,TN,Tennessee,Under $1,39580,22400,14440,980,60360,8230,19090,-2747555,29090,-2710342,90,348
19753,TN,Tennessee,"Between 1 and $10,000",439770,345410,39920,49300,443540,108380,74190,2366417,439780,2441687,4830,12132


#### Creating a new df that aggregates by county to get the totals for each county

In [8]:
income_county_agg = tn_income.groupby('county').agg('sum').reset_index()
income_county_agg.head(2)

,county,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp
0,Anderson County,34290,14990,14030,4550,65950,19620,9670,1807309,34140,1830482,700,2452
1,Bedford County,20920,8600,8010,3910,43550,15790,4430,971152,20840,985909,410,1023


In [9]:
income_county_agg['avg_income'] = round(income_county_agg.total_inc_amt * 1000 / income_county_agg.returns_with_total_inc, 0)
income_county_agg.head(3)

,county,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp,avg_income
0,Anderson County,34290,14990,14030,4550,65950,19620,9670,1807309,34140,1830482,700,2452,53617.0
1,Bedford County,20920,8600,8010,3910,43550,15790,4430,971152,20840,985909,410,1023,47308.0
2,Benton County,6610,2660,2840,1000,13020,3890,2000,269227,6580,272971,360,1119,41485.0


### Create a merged DataFrame for Heart Attack Costs and Income, keeping just `county`, `urban`, `analysis_value`, and `avg_income`; then do the same for Cancer Costs

In [10]:
# we only need the county and the average income from income_county_agg
county_incomes = income_county_agg[['county', 'avg_income']]
county_incomes.head(2)

,county,avg_income
0,Anderson County,53617.0
1,Bedford County,47308.0


In [11]:
# we only need county, urban, and analysis_value columns from the heart attack costs
tn_ha_costs = tn_ha_costs[['county', 'urban', 'analysis_value']]

In [12]:
tn_ha_costs2 = pd.merge(tn_ha_costs, county_incomes, on= 'county', how = 'left')
tn_ha_costs2.head(2)

,county,urban,analysis_value,avg_income
0,Anderson County,Urban,42749,53617.0
1,Bedford County,Rural,43661,47308.0


In [13]:
tn_ha_costs2['cost_income_ratio'] = tn_ha_costs2.analysis_value / tn_ha_costs2.avg_income
tn_ha_costs2.describe()

,analysis_value,avg_income,cost_income_ratio
count,91.000000,91.000000,91.000000
mean,45762.989011,48059.109890,0.986273
std,6574.670462,11859.019531,0.214552
min,30831.000000,35658.000000,0.473130
25%,41105.500000,42358.500000,0.850326
50%,45384.000000,44666.000000,1.001318
75%,49698.500000,50494.500000,1.117770
max,62641.000000,130072.000000,1.618290


In [14]:
# we only need county, urban, and analysis_value columns from the cancer costs
tn_cancer_costs = tn_cancer_costs[['county', 'urban', 'analysis_value']]

In [15]:
tn_cancer_costs2 = pd.merge(tn_cancer_costs, county_incomes, on= 'county', how = 'left')
tn_cancer_costs2.head(2)

,county,urban,analysis_value,avg_income
0,Anderson County,Urban,15454,53617.0
1,Bedford County,Rural,19219,47308.0


In [16]:
tn_cancer_costs2['cost_income_ratio'] = tn_cancer_costs2.analysis_value / tn_cancer_costs2.avg_income
tn_cancer_costs2.describe()

,analysis_value,avg_income,cost_income_ratio
count,95.000000,95.000000,95.000000
mean,19757.894737,47854.631579,0.430497
std,2436.304533,11760.400939,0.097032
min,15454.000000,32717.000000,0.134748
25%,18103.500000,42051.500000,0.362230
50%,19563.000000,44666.000000,0.428809
75%,21153.000000,50494.500000,0.480751
max,27740.000000,130072.000000,0.688174


### Week 5 Coding Tasks

In [17]:
#### logistic regression model for myocardial infarction costs

- create target column (1 for cost-income ratio above the mean 0 if at or below the mean)
- encode the urban column
- split train/test
- use urban column to predict

SyntaxError: invalid syntax (<ipython-input-17-38af701355e9>, line 3)

In [19]:
# create target variable
ha_cost_income_ratio_mean = tn_ha_costs2.cost_income_ratio.mean()
tn_ha_costs2['cost_ratio_above_mean'] = [1 if ratio > ha_cost_income_ratio_mean else 0 for ratio in tn_ha_costs2.cost_income_ratio]

In [20]:
tn_ha_costs2.cost_ratio_above_mean.value_counts(normalize = True)

1    0.527473
0    0.472527
Name: cost_ratio_above_mean, dtype: float64

In [21]:
tn_ha_costs2.head(2)

,county,urban,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean
0,Anderson County,Urban,42749,53617.0,0.797303,0
1,Bedford County,Rural,43661,47308.0,0.922909,0


In [22]:
# encode urban/rural
tn_ha_costs2 = pd.get_dummies(tn_ha_costs2, columns = ['urban'], drop_first = True)
tn_ha_costs2.head(2)

,county,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean,urban_Urban
0,Anderson County,42749,53617.0,0.797303,0,1
1,Bedford County,43661,47308.0,0.922909,0,0


In [23]:
#one bracket slices the column, df and 2 brackets subsets, series
X = tn_ha_costs2[['urban_Urban']]
y = tn_ha_costs2.cost_ratio_above_mean

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)

In [25]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

C:\Users\shress1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
y_pred = logistic_model.predict(X_test)

In [27]:
print(metrics.accuracy_score(y_test, y_pred))

0.6956521739130435


#### the naive model (predicting the cost-income ratio above the mean for all cases) would have accuracy .527473

In [28]:
print('                 Pred Below Mean:  Pred Above Mean:')
print('    Actual Below Mean:    ', metrics.confusion_matrix(y_test, y_pred)[0])
print('    Actual Above Mean:   ', metrics.confusion_matrix(y_test, y_pred)[1])

                 Pred Below Mean:  Pred Above Mean:
    Actual Below Mean:     [7 4]
    Actual Above Mean:    [3 9]


In [29]:
y_pred_prob = logistic_model.predict_proba(X_test)[:,1]
print('Area Under Curve:', metrics.roc_auc_score(y_test, y_pred_prob))

Area Under Curve: 0.6931818181818181


#### Let's add another predictor - the Health Factors z-score from the county health rankings: [Robert Wood Johnson Foundation](https://www.countyhealthrankings.org)

![health factors](../data/health_factors.png)

In [32]:
health_rankings = pd.read_excel('../data/2018 County Health Rankings Tennessee Data - v3.xls', 
                                sheet_name = 'Outcomes & Factors Rankings',
                               header = [0,1])

In [33]:
health_rankings.head(2)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Health Outcomes  \
                FIPS              State             County         Z-Score   
0              47000          Tennessee                NaN             NaN   
1              47001          Tennessee           Anderson        -0.26765   

        Health Factors        
   Rank        Z-Score  Rank  
0   NaN            NaN   NaN  
1  34.0      -0.423951  15.0

In [34]:
health_rankings.columns = ['fips', 'state', 'county', 'outcomes_z_score', 'outcomes_rank', 'factors_z_score', 'factors_rank']
health_rankings.head()

,fips,state,county,outcomes_z_score,outcomes_rank,factors_z_score,factors_rank
0,47000,Tennessee,NaN,NaN,NaN,NaN,NaN
1,47001,Tennessee,Anderson,-0.267650,34.0,-0.423951,15.0
2,47003,Tennessee,Bedford,-0.162724,38.0,0.065383,54.0
3,47005,Tennessee,Benton,0.976719,84.0,0.438683,83.0
4,47007,Tennessee,Bledsoe,-0.797663,15.0,0.825123,92.0


In [35]:
health_factors = health_rankings[['county', 'factors_z_score']]

#### let's see if `health_factors` and `tn_ha_costs2` can be merged as is

In [36]:
tn_ha_costs2.head(2)

,county,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean,urban_Urban
0,Anderson County,42749,53617.0,0.797303,0,1
1,Bedford County,43661,47308.0,0.922909,0,0


In [37]:
tn_ha_costs2.county = tn_ha_costs2.county.str[0:-7]

In [38]:
tn_ha_costs2.head(2)

,county,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean,urban_Urban
0,Anderson,42749,53617.0,0.797303,0,1
1,Bedford,43661,47308.0,0.922909,0,0


In [39]:
ha_with_health_factors =pd.merge(tn_ha_costs2, health_factors, on = 'county', how = 'left')
ha_with_health_factors.head(2)

,county,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean,urban_Urban,factors_z_score
0,Anderson,42749,53617.0,0.797303,0,1,-0.423951
1,Bedford,43661,47308.0,0.922909,0,0,0.065383


In [40]:
X = ha_with_health_factors[['urban_Urban', 'factors_z_score']]
y = tn_ha_costs2.cost_ratio_above_mean

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

C:\Users\shress1\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [42]:
y_pred = logistic_model.predict(X_test)

In [43]:
print(metrics.accuracy_score(y_test, y_pred))

0.7391304347826086


In [44]:
print('                 Pred Below Mean:  Pred Above Mean:')
print('    Actual Below Mean:    ', metrics.confusion_matrix(y_test, y_pred)[0])
print('    Actual Above Mean:   ', metrics.confusion_matrix(y_test, y_pred)[1])

                 Pred Below Mean:  Pred Above Mean:
    Actual Below Mean:     [7 4]
    Actual Above Mean:    [ 2 10]
